In [ ]:
%pip install gradio

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForCausalLM
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("FredZhang7/distilgpt2-stable-diffusion-v2")#"Gustavosta/MagicPrompt-Stable-Diffusion") # "FredZhang7/distilgpt2-stable-diffusion-v2")

# tokenizer = AutoTokenizer.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion")
# model = AutoModelForCausalLM.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion")

def clean_prompts(input_prompts):
    prompts=[]
    tokenizer_ner = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
    model_ner = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

    nlp = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner)
    
    for text in input_prompts:
        ner_results = nlp(text)
        names=[]
        for result in ner_results:
            names.append(result['word'])
        output_text=""
        elements = text.split(',')
        # print(elements)
        for element in elements:
            if (element.find('by') == -1) and (element.find('style of') == -1):# and not any(e in element for e in names):
                output_text += f",{element}"
        prompts.append(output_text[1:])
    return prompts

/home/ec2-user/SageMaker/video-generation-Deforum/.myenv/miniconda/envs/deforum/lib/python3.9/site-packages/transformers/modeling_utils.py:402: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(checkpoint_file, framework="pt") as f:
/home/ec2-user/SageMaker/video-generation-Deforum/.myenv/miniconda/envs/deforum/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/ec2-user/SageMaker/video-generation-Deforum/.myen

In [16]:
prompt="woman dancing alone in a room, with the music blasting from a speaker"
input_ids = tokenizer(prompt, return_tensors='pt').input_ids
if input_ids.shape[1] == 0:
    input_ids = torch.asarray(
        [[current.tokenizer.bos_token_id]], dtype=torch.long)
temperature = 1
top_k = 12
min_length = 40
max_length = 90
repetition_penalty=1
num_return_sequences=5
output = model.generate(input_ids,
                do_sample=True,
                temperature=max(float(temperature), 1e-6),
                top_k=round(top_k),
                max_length=max_length,
                num_return_sequences=num_return_sequences,
                repetition_penalty=float(repetition_penalty),
                pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
                       )
texts = tokenizer.batch_decode(output, skip_special_tokens=True)
clean_prompts(texts)

['woman dancing alone in a room, with the music blasting from a speaker-sustacean spaceship, masterpiece, award-winning, sharp focus, intricate concept art, ambient lighting, 8k, octane render, cgsociety, trending on artstation, artstationhd, artstationhq, unreal engine, 4k',
 'woman dancing alone in a room, with the music blasting from a speaker in the back, wearing a long coat, very beautiful, intricate, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, 8 k, unreal engine 5, global illumination, radiant light, detailed and intricate',
 'woman dancing alone in a room, with the music blasting from a speaker, with a red dress, wlop, ilya kuvshinov, artgerm, krenz cushart, greg rutkowski, hiroaki samura, range murata, james jean, katsuhiro otomo, erik jones, serov, surikov,',
 'woman dancing alone in a room, with the music blasting from a speaker music video, canon 5d 50 mm lens, artstation, deviantart, artstation hq, film still, detailed face